In [3]:
pip install botorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.6/629.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.w

In [5]:
import numpy as np
import pickle
import pandas as pd
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll
from scipy.stats import norm
from botorch.acquisition.analytic import ExpectedImprovement
import matplotlib.pyplot as plt
import sys
import time
import os
from torch.optim import Adam
import gpytorch
import seaborn as sns

In [138]:
data=pd.read_csv('MOF_data.csv')

In [190]:
data.head()
# Identify rows with any NaN values
rows_with_nan = data.isnull().any(axis=1)

# Display rows that contain NaN values
print("\nRows with NaN values:")
print(data[rows_with_nan])

# Remove rows with any NaN values
df = data.dropna()
df = df[df['del_q'] >= 0]
df=df.drop('Pu',axis=1) #It contains all values as 0
print("\nDataFrame after removing rows with NaN values:")
print(df)


Rows with NaN values:
                                               MOF  uptake_ads  uptake_des  \
7181  acs.inorgchem.6b00276_ic6b00276_si_006_clean    0.421515    0.395023   
7182  acs.inorgchem.6b01762_ic6b01762_si_002_clean    0.087637    0.074696   
7183  acs.inorgchem.6b02696_ic6b02696_si_002_clean    0.294982    0.280449   
7184              acscombsci.5b00188_5036495_clean    0.549701    0.613880   

         del_q  heat_ads  heat_des  del_H_ads      LISD     LFSD     LISFS  \
7181  0.026492 -39.33925       NaN        NaN  14.12459  8.71298  14.12459   
7182  0.012940 -30.35526       NaN        NaN  11.40609  8.52606  11.40609   
7183  0.014533 -34.78612       NaN        NaN  12.11622  5.03656  12.11622   
7184 -0.064179 -18.93596       NaN        NaN  11.02841  8.71025  10.83963   

      ...  La  Cr  Ti  Ba  Rh  Ce  Cu  Al  Re       COP  
7181  ...   0   0   0   0   0   0   0   0   0  0.216845  
7182  ...   0   0   0   0   0   0   0   0   0  0.105920  
7183  ...   0   0   0

In [191]:
X=df.iloc[:,7:-1]
Y=df.iloc[:,-1]
#material_names=df.iloc[:,0]

In [192]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [193]:
X_npArray=X.to_numpy()
Y_npArray=Y.to_numpy()

In [194]:
# Initialize BO parameters
nb_iterations = 110
nb_runs = 100
initial_mofs = 100  # Number of initial MOFs to randomly select

In [195]:
# Convert to torch tensors
X_tensor = torch.from_numpy(X_npArray).float()
y_tensor = torch.from_numpy(Y_npArray).float()
# Placeholder for acquired MOFs
acquired_mofs = []

# Select initial MOFs randomly
initial_ids = np.random.choice(np.arange(len(X)), size=initial_mofs, replace=False)
#initial_ids = np.random.choice(np.arange(1,100), size=initial_mofs, replace=False),
'''if taken random from only initial mofs, properties of last mofs are not being able to be predicted properly'''
acquired_mofs.extend(initial_ids)

X_unsqueezed = X_tensor.unsqueeze(1)

#features=df.columns[7:-1]

In [196]:
X_tensor = X_tensor.double()
y_tensor = y_tensor.double()

# Min-max scaling for X_train
scaler_X = MinMaxScaler()
X_tensor = torch.tensor(scaler_X.fit_transform(X_tensor), dtype=torch.float64)

# Standardization for y_train
scaler_y = StandardScaler()
y_train_reshaped = y_tensor.reshape(-1, 1)

# Fit and transform y_train
y_train_scaled = scaler_y.fit_transform(y_train_reshaped)

# Convert back to tensor
y_tensor = torch.tensor(y_train_scaled, dtype=torch.float64)

In [197]:

for i in range(initial_mofs, nb_iterations):
    print("Iteration:", i, end="\r")

    # Construct and fit GP model using currently acquired MOFs
    X_train = X_tensor[acquired_mofs, :]
    y_train = y_tensor[acquired_mofs]

    model = SingleTaskGP(X_train, y_train)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # Set up acquisition function
    acquisition_function = ExpectedImprovement(model, best_f=y_train.max().item())
    with torch.no_grad():
        acquisition_values = acquisition_function(X_unsqueezed).squeeze()

    # Select MOF with the highest acquisition value that hasn't been acquired yet
    id_max_acquisition = acquisition_values.argmax().item()
    while id_max_acquisition in acquired_mofs:
        acquisition_values[id_max_acquisition] = float('-inf')
        id_max_acquisition = acquisition_values.argmax().item()

    # Acquire this MOF
    acquired_mofs.append(id_max_acquisition)

    # Debug information
    print(f"Iteration {i}: Acquired MOF {id_max_acquisition}")
    print(f"Acquired MOFs: {acquired_mofs}")
    print(f"Expected length: {i + 1}, Actual length: {len(acquired_mofs)}")

    # Check the length of acquired_mofs
    assert len(acquired_mofs) == i + 1

print("Acquisition process completed successfully.")

/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([1.0056], dtype=torch.float64), mean = tensor([-0.0287], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


Iteration 100: Acquired MOF 0
Acquired MOFs: [3745, 6057, 1629, 2892, 4053, 5772, 95, 6658, 5955, 6025, 1421, 911, 1976, 1278, 4953, 82, 5710, 4255, 1391, 5823, 2422, 3222, 6329, 4586, 930, 1387, 4418, 993, 2302, 3370, 3508, 277, 3211, 2233, 3787, 5538, 417, 6366, 1470, 4909, 4757, 5701, 6361, 3511, 2633, 2999, 4198, 1922, 3201, 2499, 5572, 301, 1366, 6719, 1553, 4425, 3810, 6833, 5469, 5986, 6394, 1827, 3177, 4631, 737, 46, 2493, 1678, 3183, 3246, 5994, 4468, 5905, 4637, 3134, 6878, 3236, 6479, 6368, 6171, 5641, 2807, 5073, 1683, 5104, 2865, 2639, 1349, 1944, 2172, 4197, 241, 1180, 2829, 1035, 5154, 4424, 1324, 4193, 3584, 0]
Expected length: 101, Actual length: 101


/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([1.1542], dtype=torch.float64), mean = tensor([0.0286], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


Iteration 101: Acquired MOF 1
Acquired MOFs: [3745, 6057, 1629, 2892, 4053, 5772, 95, 6658, 5955, 6025, 1421, 911, 1976, 1278, 4953, 82, 5710, 4255, 1391, 5823, 2422, 3222, 6329, 4586, 930, 1387, 4418, 993, 2302, 3370, 3508, 277, 3211, 2233, 3787, 5538, 417, 6366, 1470, 4909, 4757, 5701, 6361, 3511, 2633, 2999, 4198, 1922, 3201, 2499, 5572, 301, 1366, 6719, 1553, 4425, 3810, 6833, 5469, 5986, 6394, 1827, 3177, 4631, 737, 46, 2493, 1678, 3183, 3246, 5994, 4468, 5905, 4637, 3134, 6878, 3236, 6479, 6368, 6171, 5641, 2807, 5073, 1683, 5104, 2865, 2639, 1349, 1944, 2172, 4197, 241, 1180, 2829, 1035, 5154, 4424, 1324, 4193, 3584, 0, 1]
Expected length: 102, Actual length: 102


/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([1.2595], dtype=torch.float64), mean = tensor([0.0798], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


...

Iteration 107: Acquired MOF 4
Acquired MOFs: [3745, 6057, 1629, 2892, 4053, 5772, 95, 6658, 5955, 6025, 1421, 911, 1976, 1278, 4953, 82, 5710, 4255, 1391, 5823, 2422, 3222, 6329, 4586, 930, 1387, 4418, 993, 2302, 3370, 3508, 277, 3211, 2233, 3787, 5538, 417, 6366, 1470, 4909, 4757, 5701, 6361, 3511, 2633, 2999, 4198, 1922, 3201, 2499, 5572, 301, 1366, 6719, 1553, 4425, 3810, 6833, 5469, 5986, 6394, 1827, 3177, 4631, 737, 46, 2493, 1678, 3183, 3246, 5994, 4468, 5905, 4637, 3134, 6878, 3236, 6479, 6368, 6171, 5641, 2807, 5073, 1683, 5104, 2865, 2639, 1349, 1944, 2172, 4197, 241, 1180, 2829, 1035, 5154, 4424, 1324, 4193, 3584, 0, 1, 5, 3, 6, 2, 8, 4]
Expected length: 108, Actual length: 108


/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([1.6298], dtype=torch.float64), mean = tensor([0.3393], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


Iteration 108: Acquired MOF 7
Acquired MOFs: [3745, 6057, 1629, 2892, 4053, 5772, 95, 6658, 5955, 6025, 1421, 911, 1976, 1278, 4953, 82, 5710, 4255, 1391, 5823, 2422, 3222, 6329, 4586, 930, 1387, 4418, 993, 2302, 3370, 3508, 277, 3211, 2233, 3787, 5538, 417, 6366, 1470, 4909, 4757, 5701, 6361, 3511, 2633, 2999, 4198, 1922, 3201, 2499, 5572, 301, 1366, 6719, 1553, 4425, 3810, 6833, 5469, 5986, 6394, 1827, 3177, 4631, 737, 46, 2493, 1678, 3183, 3246, 5994, 4468, 5905, 4637, 3134, 6878, 3236, 6479, 6368, 6171, 5641, 2807, 5073, 1683, 5104, 2865, 2639, 1349, 1944, 2172, 4197, 241, 1180, 2829, 1035, 5154, 4424, 1324, 4193, 3584, 0, 1, 5, 3, 6, 2, 8, 4, 7]
Expected length: 109, Actual length: 109


/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([1.6677], dtype=torch.float64), mean = tensor([0.3763], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


Iteration 109: Acquired MOF 9
Acquired MOFs: [3745, 6057, 1629, 2892, 4053, 5772, 95, 6658, 5955, 6025, 1421, 911, 1976, 1278, 4953, 82, 5710, 4255, 1391, 5823, 2422, 3222, 6329, 4586, 930, 1387, 4418, 993, 2302, 3370, 3508, 277, 3211, 2233, 3787, 5538, 417, 6366, 1470, 4909, 4757, 5701, 6361, 3511, 2633, 2999, 4198, 1922, 3201, 2499, 5572, 301, 1366, 6719, 1553, 4425, 3810, 6833, 5469, 5986, 6394, 1827, 3177, 4631, 737, 46, 2493, 1678, 3183, 3246, 5994, 4468, 5905, 4637, 3134, 6878, 3236, 6479, 6368, 6171, 5641, 2807, 5073, 1683, 5104, 2865, 2639, 1349, 1944, 2172, 4197, 241, 1180, 2829, 1035, 5154, 4424, 1324, 4193, 3584, 0, 1, 5, 3, 6, 2, 8, 4, 7, 9]
Expected length: 110, Actual length: 110
Acquisition process completed successfully.


In [198]:
#Removing features that have very strong correlations among themselves(as they compress the useful features)
df.drop(columns=["LFSD","LISFS","Di","Df","Dif","Unit_cell_volume","Channel_surface_area_A2","NASA_m2_per_cm3","NAV_A3","Channel_volume","Pocket_surface_area_A2"],inplace=True)

In [199]:
df_corr=df.iloc[:,7:]
corr_matrix=df_corr.corr()
plt.figure(figsize=(100,100))
#sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

# Setting diagonal elements of correlation matrix to 0, for comparison from rest, because correlation from self is 1
np.fill_diagonal(corr_matrix.values, 0)

threshold=0.2
# Finding out columns with correlation below the threshold
low_corr_columns = corr_matrix.columns[(abs(corr_matrix) < threshold).all()]

# Removing the low correlation columns
df_new = df.drop(low_corr_columns, axis=1)
print("Removed columns:", low_corr_columns.tolist())

Removed columns: ['NAV_cm3_per_g', 'Yb', 'Ir', 'K', 'V', 'Te', 'Sr', 'Bi', 'Mg', 'Be', 'F', 'Pb', 'Er', 'Y', 'Gd', 'Sn', 'Cs', 'B', 'Si', 'Pt', 'Th', 'Ca', 'Dy', 'Au', 'Rb', 'Zr', 'Ho', 'Br', 'Sc', 'Na', 'Pd', 'Np', 'Ru', 'Li', 'Lu', 'Hg', 'Hf', 'Eu', 'Tb', 'Tm', 'Sm', 'La', 'Cr', 'Ti', 'Ba', 'Rh', 'Ce', 'Al']


<Figure size 10000x10000 with 0 Axes>

In [200]:
cols=df_new.columns
#X=df_new.iloc[:,7:-1]
Y=df_new.iloc[:,-1]

In [201]:
columns_to_remove = ['MOF', 'uptake_ads', 'uptake_des', 'del_q', 'heat_ads', 'heat_des', 'del_H_ads']

# Remove columns from df
df_cleaned = df.drop(columns=columns_to_remove)
scaler=StandardScaler()
X=scaler.fit_transform(df_cleaned)
X = pd.DataFrame(X, columns=df_cleaned.columns)

In [202]:
Y

0       0.583945
1       0.542431
2       0.522907
3       0.515395
4       0.512710
          ...   
6912    0.000327
6913    0.000246
6914    0.000137
6915    0.000066
6916    0.000000
Name: COP, Length: 6917, dtype: float64

In [203]:
train_X=X.iloc[acquired_mofs]
train_Y=Y.iloc[acquired_mofs]
test_X=X.drop(acquired_mofs)
test_Y=Y.drop(acquired_mofs)

In [204]:
pip install scikit-optimize

In [205]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [206]:
search_spaces = {
    'n_estimators': Integer(10, 1000),  # Range for n_estimators from 300 to 1000
    'max_features': Real(0.6, 1.0),      # Fraction of features to consider for best split
    'max_depth': Integer(1, 50),         # Maximum depth of the tree
    'min_samples_split': Integer(2, 20), # Minimum number of samples required to split an internal node
    'min_samples_leaf': Integer(1, 20),  # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]           # Whether bootstrap samples are used when building trees
}

In [207]:
rfr = RandomForestRegressor()

# BayesSearchCV
opt = BayesSearchCV(
    estimator=rfr,
    search_spaces=search_spaces,
    n_iter=50,  # Number of iterations for optimization
    cv=5,  # 5-fold cross-validation
    random_state=42,
    scoring='r2'
)

In [208]:
opt.fit(train_X,train_Y)

BayesSearchCV(cv=5, estimator=RandomForestRegressor(), random_state=42,
              scoring='r2',
              search_spaces={'bootstrap': [True, False],
                             'max_depth': Integer(low=1, high=50, prior='uniform', transform='normalize'),
                             'max_features': Real(low=0.6, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=20, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=10, high=1000, prior='uniform', transform='normalize')})

In [209]:
best_model = opt.best_estimator_
y_pred = best_model.predict(test_X)
r2 = r2_score(test_Y, y_pred)
print(f"R-squared (R2): {r2}")

R-squared (R2): 0.9856434559988855


In [210]:
test_Y

10      0.477825
11      0.470049
12      0.468492
13      0.473648
14      0.471021
          ...   
6912    0.000327
6913    0.000246
6914    0.000137
6915    0.000066
6916    0.000000
Name: COP, Length: 6807, dtype: float64

In [211]:
y_pred

array([0.48547291, 0.46594697, 0.47941117, ..., 0.00746492, 0.00518328,
       0.00844229])

In [128]:
test_X

,LISD,Density,ASA_A2,ASA_m2_per_cm3,ASA_m2_per_g,NASA_A2,NASA_m2_per_g,Number_of_channels,Number_of_pockets,AV_A3,...,O,La,Cr,Ti,Ba,Rh,Ce,Cu,Al,Re
77,2.789474,-1.639610,0.072660,0.717761,2.329555,-0.137411,-0.215622,-0.548752,-0.283698,0.350618,...,-0.370844,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
80,4.109944,-1.737399,6.004034,0.903613,2.967586,-0.137411,-0.215622,-0.548752,-0.283698,5.854779,...,2.098153,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
82,2.342759,-1.586636,9.500082,0.975322,2.434864,-0.137411,-0.215622,-0.548752,-0.283698,8.382262,...,2.591952,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
83,4.397052,-1.756506,1.983855,0.426255,2.422770,-0.137411,-0.215622,-0.548752,-0.283698,2.835453,...,0.122955,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,1.123520,-0.079596,-0.040141
84,1.454016,-1.648002,1.153385,0.384081,1.963016,-0.137411,-0.215622,-0.548752,-0.283698,1.619621,...,0.073576,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6912,-0.446375,0.963847,-0.408659,-0.554974,-0.760180,-0.137411,-0.215622,0.371645,-0.283698,-0.264871,...,-0.074564,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
6913,-0.668863,3.743522,-0.544968,-1.615648,-1.227891,0.274519,1.662517,-0.548752,0.065473,-0.293707,...,-0.346154,-0.100108,-0.043459,-0.029774,3.220348,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
6914,-0.860054,0.892053,-0.561349,-1.654813,-1.119637,-0.137411,-0.215622,-0.548752,-0.283698,-0.296749,...,-0.667123,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,-0.271953,-0.079596,-0.040141
6915,-0.795200,1.915823,-0.232247,-1.621076,-1.168881,-0.137411,-0.215622,2.212438,-0.283698,-0.255103,...,-0.667123,-0.100108,-0.043459,-0.029774,-0.051079,-0.022009,-0.081034,5.309938,-0.079596,-0.040141
